# <center>Классификация рукописных цифр

In [1]:
import keras
from keras.datasets import mnist

Загрузим обучающую и тестовую выборки.

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Для распознавания рукописных цифр будем использовать полносвязную нейронную сеть, где каждый нейрон входного слоя характеризует один пиксель исходного изображения. При этом сами изображения представляют собой матрицу чисел, поэтому для дальнейшей работы необходимо выполнить преобразования над данными.

С помощью функции `reshape` преобразуйте каждую матрицу в вектор длины 28*28 (размер исходных изображений).

In [3]:
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

Поскольку мы будем использовать градиентные методы обучения, необходимо также нормализовать данные.

In [4]:
x_train = x_train / 255
x_test = x_test / 255

В данный момент значение целевой переменной каждого изображения представляет собой цифру от 0 до 9. Для решения задачи многоклассовой классификации можно использовать функцию softmax. Её выход при этом - вектор вероятностей принадлежности к каждому из классов. Таким образом, возникает необходимость преобразовать `y_train` и `y_test`.

Для этих целей в `keras` доступна функция `keras.utils.to_categorical`, принимающая на вход исходный вектор значений и количество классов. Примените данную функцию к  `y_train` и  `y_test`.

In [5]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

Теперь можно приступать к обучению сети. Будем использовать следующую архитектуру:
- на вход подается 784 значения;
- далее идет полносвязный слой с 256 нейронами и функцией активации `relu`;
- для повышения обобщающей способности модели добавим `Dropout(0.2)`, где `0.2` - вероятность отключения каждого из нейронов слоя;
- далее еще один полносвязный слой с 256 нейронами и функцией активации `relu`;
- последний слой - выход сети - полносвязный слой с числом нейронов, равным числу классов и функцией активации `softmax`.

In [6]:
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten

In [7]:
model = Sequential([ 
    Dense(256, activation='relu'), 
    Dropout(0.2), 
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

In [8]:
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )

Обучите полученную модель на обучающей выборке. Для обучения:
- выберите нужную функцию потерь: https://keras.io/losses/; 
- optimizer - RMSProp: https://keras.io/optimizers/; 
- размер мини-батча - 128;
- число эпох - 10;
- на каждом шаге выводите значение `accuracy` как на обучающей, так и на тестовой выборках.

In [9]:
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/5
469/469 [==============================] - 2s 5ms/step - loss: 0.2762 - accuracy: 0.9169 - val_loss: 0.1256 - val_accuracy: 0.9600
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1201 - accuracy: 0.9629 - val_loss: 0.0914 - val_accuracy: 0.9720
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0887 - accuracy: 0.9730 - val_loss: 0.0790 - val_accuracy: 0.9751
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0709 - accuracy: 0.9780 - val_loss: 0.0774 - val_accuracy: 0.9757
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0603 - accuracy: 0.9811 - val_loss: 0.0721 - val_accuracy: 0.9781


In [10]:
y_pred = model.predict(x_test)

In [11]:
import numpy as np

In [12]:
np.mean(keras.losses.categorical_crossentropy(y_test, y_pred))

0.07206122

In [13]:
acc_dense = 1 - np.abs(y_test - y_pred).mean()
acc_dense

0.9943114654161036

Попробуйте улучшить полученный результат, меняя архитектуру сети.

In [14]:
import numpy as np

In [15]:
import keras
from keras.datasets import mnist

In [16]:
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten

In [17]:
from keras.layers import Conv2D, MaxPooling2D, Dropout

In [18]:
model_cnv = Sequential([
    Conv2D(16, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), strides=(2, 2), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
#     Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')]
)

In [19]:
model_cnv.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [21]:
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [22]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [23]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [24]:
print(y_train.shape, y_test.shape)

(60000, 10) (10000, 10)


In [25]:
model_cnv.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/5
469/469 [==============================] - 11s 23ms/step - loss: 1.4133 - accuracy: 0.7942 - val_loss: 0.2682 - val_accuracy: 0.9214
Epoch 2/5
469/469 [==============================] - 11s 24ms/step - loss: 0.2269 - accuracy: 0.9337 - val_loss: 0.1741 - val_accuracy: 0.9472
Epoch 3/5
469/469 [==============================] - 11s 23ms/step - loss: 0.1555 - accuracy: 0.9540 - val_loss: 0.1294 - val_accuracy: 0.9612
Epoch 4/5
469/469 [==============================] - 11s 23ms/step - loss: 0.1215 - accuracy: 0.9632 - val_loss: 0.1186 - val_accuracy: 0.9653
Epoch 5/5
469/469 [==============================] - 11s 23ms/step - loss: 0.1037 - accuracy: 0.9680 - val_loss: 0.1193 - val_accuracy: 0.9654


In [26]:
y_pred = model_cnv.predict(x_test)
print('Точность свёрточной сети:  \t{:.4f}'.format(1 - np.abs(y_test - y_pred).mean()), 
      '\nТочность полносвязной сети:\t{:.4f}'.format(acc_dense))

Точность свёрточной сети:  	0.9911 
Точность полносвязной сети:	0.9943


Слава богу